In [5]:
'''
게시글 종류에는 이미지 게시글, 코스 소개게시글, 동적 이미지게시글 존재. 동적 이미지글은 추가중. 코스게시글은 카드요소를 클릭해
각 내부 이미지 추출. 카드요소는 4개로 고정함. 
'''

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import urllib.request
import urllib
import time
import math
import os
import random

In [19]:
#[크롤링 키워드입력]
print('='*80)
print('예제:대한민국 구석구석 사이트의 여행지 정보 수집하기')
print('='*80)
query_txt = input('크롤링할 키워드를 입력하세요:')
cnt = int(input('크롤링할 건수는 몇건입니까?'))
page_cnt = math.ceil(cnt/10) #결과 올림

예제:대한민국 구석구석 사이트의 여행지 정보 수집하기
크롤링할 키워드를 입력하세요:정동진해변
크롤링할 건수는 몇건입니까?30


In [22]:
#[사진폴더지정,브라우저 열기]
f_dir= input('사진을 저장할 폴더를 지정하세요(예:C:\\study\\python\\notebook\\data\\):')

now = time.localtime()
s = "{}-{}-{}-{}-{}-{}".format(now.tm_year,now.tm_mon,now.tm_mday,now.tm_hour,now.tm_min,now.tm_sec)

#총 이미지 다운할 폴더 생성
os.makedirs(f_dir+s+'-'+query_txt)
r_result_dir = f_dir+s+'-'+query_txt

#브라우저 열기
path = "c:/study/python/datadown/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get('https://korean.visitkorea.or.kr')
driver.maximize_window()
time.sleep(random.randrange(2,5)) #2에서 5사이 랜덤시간실행
try:
    driver.find_element_by_xpath('//*[@id="chkForm01"]').click()
except:
    print('코로나 창이 없습니다')

#키워드로 검색
element = driver.find_element_by_id('inp_search')
element.send_keys(query_txt)
element.send_keys('\n')

print('='*80)
print('크롤링할 키워드:',query_txt)
print('크롤링할 총 페이지 번호:',page_cnt)
print('='*80)

사진을 저장할 폴더를 지정하세요(예:C:\study\python\notebook\data\):C:\study\python\notebook\data\
크롤링할 키워드: 정동진해변
크롤링할 총 페이지 번호: 3


In [23]:
#[게시글 별 이미지 크롤링]

no2 = []
contents2 = []
no = 1

#페이지 별 반복
for x in range(1,page_cnt + 1):
    print("{}페이지 내용 수집중 =========================================================".format(x))
    
    #게시글 별 반복
    for i in range(1,11):
        if no > cnt:
            break
        file_no = 0
        #게시글클릭
        try:
            driver.find_element_by_xpath('''//*[@id="listBody"]/ul/li[{}]/div[2]/div[1]/a'''.format(i)).click()
            time.sleep(random.randrange(2,5))  
        
            #페이지 html 소스추출
            html = driver.page_source
            soup = BeautifulSoup(html,'html.parser')
            img_src2 = []

            #게시글 제목의 폴더 생성
            try:
                title = soup.find('div','tit_cont').find('h2').get_text()
                print(title +' : 이미지 추출중 =========================')
                os.makedirs(r_result_dir+'\\'+title)
            #코스 게시글 제목의 폴더 생성
            except: 
                title = soup.find('div','tit').find('h2').get_text()
                print(title +' : 코스 이미지 추출중 =========================')
                os.makedirs(r_result_dir+'\\'+title)

            #이미지 추출
            try:
                img_srcs = soup.find('div','box_txtPhoto').find_all('img')

                for i in img_srcs:
                    img_src2.append(i['src'])
            #코스 이미지 추출
            except:
                try:
                    #<<만약 카드개수가 여러개면 개수확인후 추출 추가>>
                    for y in range(1,5):
                        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[6]/div[1]/div[1]/div/ul/li[{}]/a""".format(y)).click()

                        time.sleep(2)

                        html = driver.page_source
                        soup = BeautifulSoup(html,'html.parser')
                        tag = soup.find('div','cos_cont active')
                        tag2 = tag.find('div','info_area pc').find_all('a')

                        for i in tag2:
                            style = i['style'].split('(')[1].split(')')[0] #style속성안에 괄호안에있는거
                            img_src2.append(style)
                except: 
                    driver.back()
                    continue

            #print(img_src2) 확인용

            #이미지 다운로드 
            os.chdir(r_result_dir+'\\'+title)
            
            for i in range(0,len(img_src2)):
                try:
                    urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
                except:
                    continue
                file_no += 1
                time.sleep(0.5)
                print("{} 번째 이미지 다운로드 중......".format(file_no))
            driver.back()
            time.sleep(2)
            no += 1
        except:
            driver.back()
            continue
        
        

    if x > page_cnt + 1:
        break
        
    x += 1
    time.sleep(1.5)
    
    #페이지 이동
    if(x%5 == 1):
        driver.find_element_by_link_text('다음').click()
    else:
        driver.find_element_by_link_text('{}'.format(x)).click()
    
    time.sleep(2)


1페이지 내용 수집중 =========================================================
정동진 모래시계 공원(2016년 선정 열린관광지 ) : 이미지 추출중 =========================
1 번째 이미지 다운로드 중......
2 번째 이미지 다운로드 중......
3 번째 이미지 다운로드 중......
4 번째 이미지 다운로드 중......
5 번째 이미지 다운로드 중......
6 번째 이미지 다운로드 중......
7 번째 이미지 다운로드 중......
8 번째 이미지 다운로드 중......
9 번째 이미지 다운로드 중......
10 번째 이미지 다운로드 중......
11 번째 이미지 다운로드 중......
12 번째 이미지 다운로드 중......
13 번째 이미지 다운로드 중......
14 번째 이미지 다운로드 중......
15 번째 이미지 다운로드 중......
16 번째 이미지 다운로드 중......
17 번째 이미지 다운로드 중......
18 번째 이미지 다운로드 중......
19 번째 이미지 다운로드 중......
20 번째 이미지 다운로드 중......
21 번째 이미지 다운로드 중......
22 번째 이미지 다운로드 중......
23 번째 이미지 다운로드 중......
24 번째 이미지 다운로드 중......
25 번째 이미지 다운로드 중......
26 번째 이미지 다운로드 중......
기암·항구·해변이 멋진 여름 바다로 떠나다, 주문진 아들바위 : 이미지 추출중 =========================
1 번째 이미지 다운로드 중......
2 번째 이미지 다운로드 중......
3 번째 이미지 다운로드 중......
4 번째 이미지 다운로드 중......
5 번째 이미지 다운로드 중......
6 번째 이미지 다운로드 중......
7 번째 이미지 다운로드 중......
8 번째 이미지 다운로드 중......
9 번째 이미지 다운로드 중......
10 번째 이미

1 번째 이미지 다운로드 중......
2 번째 이미지 다운로드 중......
3 번째 이미지 다운로드 중......
4 번째 이미지 다운로드 중......
5 번째 이미지 다운로드 중......
6 번째 이미지 다운로드 중......
7 번째 이미지 다운로드 중......
8 번째 이미지 다운로드 중......
9 번째 이미지 다운로드 중......
10 번째 이미지 다운로드 중......
11 번째 이미지 다운로드 중......
12 번째 이미지 다운로드 중......
13 번째 이미지 다운로드 중......
14 번째 이미지 다운로드 중......
15 번째 이미지 다운로드 중......
16 번째 이미지 다운로드 중......
17 번째 이미지 다운로드 중......
18 번째 이미지 다운로드 중......
19 번째 이미지 다운로드 중......
20 번째 이미지 다운로드 중......
21 번째 이미지 다운로드 중......
22 번째 이미지 다운로드 중......
23 번째 이미지 다운로드 중......
24 번째 이미지 다운로드 중......
25 번째 이미지 다운로드 중......
26 번째 이미지 다운로드 중......
27 번째 이미지 다운로드 중......
28 번째 이미지 다운로드 중......
29 번째 이미지 다운로드 중......
30 번째 이미지 다운로드 중......
31 번째 이미지 다운로드 중......
32 번째 이미지 다운로드 중......
33 번째 이미지 다운로드 중......
34 번째 이미지 다운로드 중......
35 번째 이미지 다운로드 중......
36 번째 이미지 다운로드 중......
37 번째 이미지 다운로드 중......
38 번째 이미지 다운로드 중......
바다 그리고 가시연꽃의 향연, 강릉 : 이미지 추출중 =========================
1 번째 이미지 다운로드 중......
2 번째 이미지 다운로드 중......
3 번째 이미지 다운로드 중......
4 번째 이미지 다운로드

In [ ]:
e_time = time.time()
t_time = e_time - s_time

print('\n')
print('='*80)
print('총 소요시간 {}초 입니다.'.format(round(t_time,1)))
print('='*80)

driver.close()